In [ ]:
import os
import mne
import numpy as np
import pandas as pd
import glob

# Exploring CHB-MIT dataset

## Reading edf files

In [ ]:
patient_id = 1
DATASET_PATH = os.path.join(os.getcwd(), 'data', 'chb01-summary.txt') #ONLY FOR TESTING
patient_id_str = str(patient_id).zfill(2)

all_file_path = glob(f'chb{patient_id_str}/*.edf')
print(len(all_file_path))

In [ ]:
raw = mne.io.read_raw_edf(all_file_path[0])
print(raw.get_channel_types())
print(raw.info)
print(raw.ch_names)
print(raw.get_data().shape)

In [ ]:
def read_data_edf(file_path):
    data = mne.io.read_raw_edf(file_path, preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5, h_freq=45)
    epochs = mne.make_fixed_length_epochs(data, duration=10, overlap=1)
    epochs_array = epochs.get_data()
    return epochs, epochs_array
# def read_data_seizures(file_path):
#     ref_file = open(file_path, 'r')
#     array=epochs.get_data()
#     return array

def read_edf_to_raw(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True)
    raw.set_eeg_reference()
    raw.filter(l_freq=0.5, h_freq=45)
    return raw

In [ ]:
%%capture

# Shape ()
epochs_data_list = [read_data_edf(i)[1] for i in all_file_path]
epochs_list = [read_data_edf(i)[0] for i in all_file_path]


epochs_data_array = np.array(epochs_data_list)
epochs_data_array.shape

# merge the files on one array 
# shape: (N_files, 399 epochs, 23 channels, 2560 values) ->  (N_files*399 epochs, 23 channels, 2560 values)
data_array = np.vstack(epochs_data_array)
data_array.shape

## Extract features

In [ ]:
########! Features extraction !#####
from scipy import stats
def mean(x):
    return np.mean(x, axis=-1)
def std(x):
    return np.std(x, axis=-1)
def ptp(x):
    return np.ptp(x, axis=-1)
def var(x):
    return np.var(x, axis=-1)
def minim(x):
    return np.min(x, axis=-1)
def maxim(x):
    return np.max(x, axis=-1)
def argminim(x):
    return np.argmin(x, axis=-1)
def argmaxim(x):
    return np.argmax(x, axis=-1)
def rms(x):
    return np.sqrt(np.mean(x**2, axis=-1))
def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis=-1)
def skewness(x):
    return stats.skew(x, axis=-1)
def kurtosis(x):
    return stats.kurtosis(x, axis=-1)
def concatenate_features(x):
    return np.concatenate((mean(x), std(x), ptp(x), var(x), minim(x), maxim(x), argmaxim(x), 
                           argminim(x), rms(x), abs_diff_signal(x), skewness(x), kurtosis(x)), axis=-1)

In [ ]:
features = []
for d in data_array:
    features.append(concatenate_features(d))
    
features_array = np.array(features)
features_array.shape

## Visualize features per channel and per epoch

In [ ]:
# get features of specific channel
def features_by_channel(features_array):
    features_channel = []
    MAX_CHANNELS = 23
    for i in range(MAX_CHANNELS):
        array = features_array[:, i*12:(i+1)*12];
        features_channel.append(pd.DataFrame(array, columns=['Mean', 'Std', 'Ptp', 'Var', 'Minimo', 'Maxim', 'Arg Max', 'Arg Min', 'RMS', 'ABS DIFF', 'skewness', 'kurtosis']))
    return features_channel

features_channel = features_by_channel(features_array)
print('Info for channel 23: ')

# Shape of features_channel=(23, 1596, 12)
features_channel[22].shape

# Preprocessing and labelling data

In [ ]:
# patient_folder = Path.cwd()/'chbmit'/'chb{}'.format(str(patient_id).zfill(2))
# DATASET_PATH = 
DATASET_PATH = os.getcwd() #ONLY FOR TESTING

patient_id = 1
patient_id_str = str(patient_id).zfill(2)

patient_folder = os.path.join(DATASET_PATH, f'chb{patient_id_str}')
print(f"Serching for patient {patient_id_str} files at {patient_folder} ...")
patient_files = glob(os.path.join(patient_folder, f'*.edf'))
print(f"Found {len(patient_files)} files")
summary_path = os.path.join(patient_folder, f'chb{patient_id_str}-summary.txt')

In [ ]:
## READ SUMMARY (TODO: automate this)
seizures_dict = {"chb01_03": [[2996, 3036]],
                "chb01_04": [[1467, 1494]],
                "chb01_15": [[1732, 1772]],
                "chb01_16": [[1015, 1066]],
                "chb01_18": [[1720, 1810]],
                "chb01_21": [[327, 420]],
                "chb01_26": [[1862, 1963]],
                
                "chb02_16": [[130, 212]],

                "chb05_06": [[417, 532]], 
                "chb05_13": [[1086, 1196]],
                "chb05_16": [[2317, 2413]], 
                "chb05_17": [[2451, 2571]],
                "chb05_22": [[2348, 2465]],
                
                "chb08_02": [[2670, 2841]], 
                "chb08_05": [[2856, 3046]],
                "chb08_11": [[2988, 3211]], 
                "chb08_13": [[2417, 2577]],
                "chb08_21": [[2083, 2347]]}

curr_time = 0
epoch_time = 10
overlap = 5

# Divide into epochs
labels = []
for file_path in patient_files:
    # get filename: chbxx_xx.edf
    filename = os.path.split(file_path)[1]

    # remove .edf staying only chbxx_xx
    filename = os.path.splitext(filename)[0]
    
    # read raw
    raw = read_edf_to_raw(file_path)

    curr_time = 0
    while curr_time <= max(raw.times) + 0.01 - epoch_time:  # max(raw.times) = 3600
        epoch_features = []
        start_time = curr_time 
        if start_time < 0.:
            start_time = 0.
        end_time = curr_time + epoch_time
        start, stop = raw.time_as_index([start_time, end_time])
        temp = raw[:, start:stop][0]

        # start time as ID
        # features.append(start_time)

        # features
        epoch_features.extend(concatenate_features(temp))

        # seizure flag for y
        aux = []
        if filename in seizures_dict:  # if file has seizure
            for seizure in seizures_dict[filename]:
                if start_time > seizure[0] and start_time < seizure[1]:
                    aux.append(1)
                elif start_time + epoch_time > seizure[0] and start_time + epoch_time < seizure[1]:
                    aux.append(1)
                else:
                    aux.append(0)
        else:    
            aux.append(0)

        if 1 in aux:
            epoch_features.extend([1])
        else:
            epoch_features.extend([0])

        labels.append(epoch_features)

        curr_time = curr_time + epoch_time - overlap  
        print("Section ", str(len(labels)), "; start: ", start, " ; stop: ", stop)

In [ ]:
labels = np.array(labels)
labels.shape

### Save to npy

In [ ]:
# np.savetxt("chb01_10i_5o_wlabel.txt", np.array(labels), delimiter=" ")
np.save("seizures_labeled", np.array(labels))